### Libraries:

In [ ]:
# !pip install sklearn-genetic-opt

In [ ]:
# !pip install scikit-optimize

In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn_genetic.space import Categorical, Continuous
from hyperopt import fmin, tpe, hp, Trials
import numpy as np
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
#from my_optimizer import ModelOptimizer

from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn import preprocessing

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### Warnings:

In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="The objective has been evaluated at this point before.")

### Read dataset:

In [3]:
import pandas as pd
df = pd.read_csv('data1_filled.csv')
df

,Study_Folder,Subject_Folder,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347,0.124451,0.166405
1,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335,0.124451,0.166405
2,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500,0.124451,0.166405
3,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129,0.124451,0.166405
4,0eb02000-fe04-480d-9da4-b60e758d46e7,videos,0,4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347,0.124451,0.166405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107845,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,200,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527,0.124735,0.721234
107846,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,201,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992,0.223992,0.539739
107847,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,202,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993,0.124735,0.733096
107848,7f1cb219-f275-4148-9658-67b1c26a6649,videos,9806,203,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297,0.124204,0.717675


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import pandas as pd
# import io

# df = pd.read_csv(io.BytesIO(uploaded['data1_filled.csv']))
# df

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107850 entries, 0 to 107849
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Study_Folder    107850 non-null  object 
 1   Subject_Folder  107850 non-null  object 
 2   Stimuli         107850 non-null  int64  
 3   Frame Number    107850 non-null  int64  
 4   Face_Y          107850 non-null  float64
 5   Face_X          107850 non-null  float64
 6   Face_W          107850 non-null  float64
 7   Face_H          107850 non-null  float64
 8   Right_Eye_X     107850 non-null  float64
 9   Right_Eye_Y     107850 non-null  float64
 10  Left_Eye_X      107850 non-null  float64
 11  Left_Eye_Y      107850 non-null  float64
 12  Face_Angle      107850 non-null  float64
 13  X_Predictions   107850 non-null  float64
 14  Y_Predictions   107850 non-null  float64
dtypes: float64(11), int64(2), object(2)
memory usage: 12.3+ MB


### Drop columns:

In [5]:
df.drop(['Study_Folder','Subject_Folder'], axis = 1, inplace = True)

In [6]:
df

,Stimuli,Frame Number,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle,X_Predictions,Y_Predictions
0,0,0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347,0.124451,0.166405
1,0,1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335,0.124451,0.166405
2,0,2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500,0.124451,0.166405
3,0,3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129,0.124451,0.166405
4,0,4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347,0.124451,0.166405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107845,9806,200,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527,0.124735,0.721234
107846,9806,201,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992,0.223992,0.539739
107847,9806,202,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993,0.124735,0.733096
107848,9806,203,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297,0.124204,0.717675


In [7]:
# Checking NaN values
df.isna().sum()

Stimuli          0
Frame Number     0
Face_Y           0
Face_X           0
Face_W           0
Face_H           0
Right_Eye_X      0
Right_Eye_Y      0
Left_Eye_X       0
Left_Eye_Y       0
Face_Angle       0
X_Predictions    0
Y_Predictions    0
dtype: int64

### Define Stimuli, X,Y:

In [8]:
df.columns

Index(['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W', 'Face_H',
       'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle',
       'X_Predictions', 'Y_Predictions'],
      dtype='object')

In [9]:
features_columns = ['Stimuli', 'Frame Number', 'Face_Y', 'Face_X', 'Face_W','Face_H',
                    'Right_Eye_X', 'Right_Eye_Y', 'Left_Eye_X', 'Left_Eye_Y', 'Face_Angle']

target_columns=['X_Predictions', 'Y_Predictions']

In [10]:
X=df[features_columns]

In [11]:
y=df[target_columns]

In [12]:
X_train_valid= X[X['Stimuli']<= 11]

In [13]:
X_test= X[X['Stimuli']> 11]

In [14]:
y_train_valid= y[X['Stimuli']<= 11]

In [15]:
y_test= y[X['Stimuli']> 11]

### Split data:

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.3,stratify =X_train_valid['Stimuli'] ,random_state=42)

In [17]:
print("Whole dataset :",len(df))
print("X Train size", len(X_train))
print("y Train size", len(y_train))
print("X Valid size", len(X_valid))
print("y Valid size", len(y_valid))
print("X Test size", len(X_test))
print("y Test size", len(y_test))

Whole dataset : 107850
X Train size 38986
y Train size 38986
X Valid size 16709
y Valid size 16709
X Test size 52155
y Test size 52155


In [18]:
print("X Train shape", X_train.shape)
print("y Train shape", y_train.shape)
print("X Valid shape", X_valid.shape)
print("y Valid shape", y_valid.shape)
print("X Test shape", X_test.shape)
print("y Test shape", y_test.shape)

X Train shape (38986, 11)
y Train shape (38986, 2)
X Valid shape (16709, 11)
y Valid shape (16709, 2)
X Test shape (52155, 11)
y Test shape (52155, 2)


In [19]:
X_train['Stimuli'].value_counts().sort_values()/len(X_train), X_valid['Stimuli'].value_counts().sort_values()/len(X_valid)

(0     0.081542
 11    0.082389
 5     0.082389
 6     0.083158
 2     0.083671
 7     0.083748
 3     0.083748
 10    0.083774
 8     0.083774
 4     0.083799
 9     0.083825
 1     0.084184
 Name: Stimuli, dtype: float64,
 0     0.081513
 5     0.082351
 11    0.082411
 6     0.083189
 2     0.083667
 10    0.083727
 7     0.083727
 8     0.083787
 3     0.083787
 4     0.083787
 9     0.083847
 1     0.084206
 Name: Stimuli, dtype: float64)

### Drop columns:

In [20]:
drop_column = ['Stimuli','Frame Number']

In [21]:
X.drop(columns = drop_column, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_12828\3791142718.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns = drop_column, inplace=True)


In [22]:
X

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
0,235.0,195.0,204.0,204.0,269.0,377.0,269.0,296.0,0.469347
1,235.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.469335
2,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.465500
3,236.0,196.0,201.0,201.0,268.0,377.0,268.0,296.0,0.477129
4,235.0,195.0,203.0,203.0,269.0,377.0,268.0,296.0,0.469347
...,...,...,...,...,...,...,...,...,...
107845,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527
107846,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992
107847,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993
107848,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297


In [23]:
X_train.drop(columns = drop_column, inplace=True)

In [24]:
X_train

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
8500,194.0,182.0,237.0,237.0,274.0,361.0,285.0,273.0,0.303291
34560,241.0,214.0,196.0,196.0,290.0,378.0,295.0,307.0,0.572514
49079,233.0,89.0,251.0,251.0,191.0,408.0,195.0,317.0,2.094841
53301,255.0,178.0,262.0,262.0,284.0,429.0,284.0,342.0,0.450649
31581,200.0,127.0,217.0,217.0,205.0,353.0,208.0,268.0,0.608209
...,...,...,...,...,...,...,...,...,...
42620,145.0,314.0,189.0,189.0,386.0,272.0,383.0,211.0,0.176726
24503,264.0,230.0,180.0,180.0,300.0,386.0,303.0,329.0,0.535068
34795,178.0,207.0,215.0,215.0,291.0,310.0,291.0,243.0,0.267951
50664,284.0,211.0,205.0,205.0,293.0,426.0,295.0,355.0,0.584136


In [25]:
X_valid.drop(columns = drop_column, inplace=True)

In [26]:
X_valid

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
1879,203.0,76.0,222.0,222.0,167.0,369.0,163.0,283.0,0.731607
54927,195.0,189.0,230.0,230.0,282.0,349.0,284.0,263.0,0.291002
26246,239.0,248.0,214.0,214.0,336.0,391.0,332.0,304.0,0.278280
51101,191.0,158.0,232.0,232.0,243.0,347.0,255.0,262.0,0.381798
54715,204.0,158.0,248.0,248.0,260.0,374.0,259.0,267.0,0.309640
...,...,...,...,...,...,...,...,...,...
32028,283.0,205.0,203.0,203.0,286.0,405.0,289.0,351.0,0.602547
23295,236.0,194.0,204.0,204.0,271.0,379.0,272.0,302.0,0.477154
43401,308.0,77.0,247.0,247.0,173.0,470.0,171.0,393.0,1.450301
49843,277.0,200.0,243.0,243.0,295.0,431.0,305.0,346.0,0.510805


In [27]:
X_test.drop(columns = drop_column, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_12828\2944433928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = drop_column, inplace=True)


In [28]:
X_test

,Face_Y,Face_X,Face_W,Face_H,Right_Eye_X,Right_Eye_Y,Left_Eye_X,Left_Eye_Y,Face_Angle
55695,241.0,220.0,194.0,194.0,288.0,379.0,288.0,297.0,0.424277
55696,242.0,221.0,193.0,193.0,288.0,379.0,288.0,297.0,0.421167
55697,242.0,221.0,193.0,193.0,287.0,379.0,287.0,297.0,0.421158
55698,241.0,221.0,193.0,193.0,287.0,379.0,287.0,297.0,0.421158
55699,240.0,221.0,195.0,195.0,288.0,379.0,287.0,296.0,0.421158
...,...,...,...,...,...,...,...,...,...
107845,200.0,162.0,233.0,233.0,243.0,353.0,256.0,270.0,0.362527
107846,201.0,160.0,233.0,233.0,243.0,353.0,255.0,270.0,0.386992
107847,200.0,162.0,232.0,232.0,244.0,352.0,256.0,270.0,0.386993
107848,200.0,159.0,236.0,236.0,243.0,353.0,255.0,270.0,0.349297


### MinMaxScaler:

In [29]:
minmax_scaler = preprocessing.MinMaxScaler()
X_train_scal = minmax_scaler.fit_transform(X_train)
X_valid_scal = minmax_scaler.transform(X_valid)
X_test_scal = minmax_scaler.transform(X_test)

In [30]:
X_train_scal

array([[0.3874092 , 0.53246753, 0.39884393, ..., 0.57638889, 0.39793282,
        0.00243009],
       [0.50121065, 0.63636364, 0.16184971, ..., 0.61111111, 0.48578811,
        0.00624757],
       [0.48184019, 0.23051948, 0.47976879, ..., 0.26388889, 0.51162791,
        0.02783358],
       ...,
       [0.34866828, 0.61363636, 0.2716763 , ..., 0.59722222, 0.32041344,
        0.00192898],
       [0.60532688, 0.62662338, 0.21387283, ..., 0.61111111, 0.60981912,
        0.00641237],
       [0.66101695, 0.18831169, 0.39306358, ..., 0.16319444, 0.67183463,
        0.01818552]])

In [31]:
X_valid_scal

array([[0.40920097, 0.18831169, 0.31213873, ..., 0.15277778, 0.42377261,
        0.00850345],
       [0.38983051, 0.55519481, 0.3583815 , ..., 0.57291667, 0.37209302,
        0.00225584],
       [0.49636804, 0.74675325, 0.26589595, ..., 0.73958333, 0.47803618,
        0.00207545],
       ...,
       [0.66343826, 0.19155844, 0.4566474 , ..., 0.18055556, 0.70801034,
        0.01869425],
       [0.58837772, 0.59090909, 0.43352601, ..., 0.64583333, 0.58656331,
        0.00537255],
       [0.3559322 , 0.23701299, 0.65895954, ..., 0.33680556, 0.37209302,
        0.00323463]])

In [32]:
X_test_scal

array([[0.50121065, 0.65584416, 0.15028902, ..., 0.58680556, 0.45994832,
        0.00414562],
       [0.50363196, 0.65909091, 0.14450867, ..., 0.58680556, 0.45994832,
        0.00410153],
       [0.50363196, 0.65909091, 0.14450867, ..., 0.58333333, 0.45994832,
        0.00410139],
       ...,
       [0.40193705, 0.46753247, 0.3699422 , ..., 0.47569444, 0.39018088,
        0.00361695],
       [0.40193705, 0.45779221, 0.39306358, ..., 0.47222222, 0.39018088,
        0.00308244],
       [0.40677966, 0.47077922, 0.3583815 , ..., 0.47569444, 0.39018088,
        0.00347246]])

### Save csv file (Train,valid,test):

In [33]:
pd.DataFrame(X_train_scal).to_csv('X_train_scal_MMS.csv', index=False)

In [34]:
pd.DataFrame(X_valid_scal).to_csv('X_valid_scal_MMS.csv', index=False)

In [35]:
pd.DataFrame(X_test_scal).to_csv('X_test_scal_MMS.csv', index=False)

In [36]:
pd.DataFrame(y_train).to_csv('y_train.csv', index=False)

In [37]:
pd.DataFrame(y_valid).to_csv('y_valid.csv', index=False)

In [38]:
pd.DataFrame(y_test).to_csv('y_test.csv', index=False)

In [ ]:
# from google.colab import files

# files.download('X_train_scal_MMS.csv')

### Function for Model Optimizer:

In [39]:
class ModelOptimizer:
    results = []

    def __init__(self, model, optimization_method='grid', optimization_params=None):
        self.model = model
        self.optimization_method = optimization_method
        self.optimization_params = optimization_params

    def optimize(self, X, y):
        if self.optimization_method == 'grid':
            return self.grid_search(X, y)
        elif self.optimization_method == 'randomized':
            return self.random_search(X, y)
        elif self.optimization_method == 'bayesian':
            return self.bayesian_optimization(X, y)
        elif self.optimization_method == 'bayesian_hyperparameters':
            return self.bayesian_optimization_hyperparameters(X, y)
        elif self.optimization_method == 'genetic':
            return self.genetic_algorithm(X, y)
        else:
            raise ValueError('Invalid optimization method.')

    def grid_search(self, X, y):
        optimizer = GridSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def random_search(self, X, y):
        optimizer = RandomizedSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def bayesian_optimization(self, X, y):
        optimizer = BayesSearchCV(self.model, self.optimization_params, cv=5)
        optimizer.fit(X, y)
        best_estimator = optimizer.best_estimator_
        best_params = optimizer.best_params_
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def bayesian_optimization_hyperparameters(self, X, y):
        space = self._convert_to_hyperopt_space(self.optimization_params)
        objective = self._create_objective_function(X, y)
        trials = Trials()
        best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)
        best_params = self._convert_hyperopt_params(best)
        best_estimator = self.model.set_params(**best_params)
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def genetic_algorithm(self, X, y):
        ga = GeneticAlgorithm(self.model, self.optimization_params)
        best_estimator, best_params = ga.optimize(X, y)
        self.results.append((best_estimator, best_params))
        return best_estimator, best_params

    def _convert_to_hyperopt_space(self, optimization_params):
        space = {}
        for param, values in optimization_params.items():
            if isinstance(values, list):
                space[param] = hp.choice(param, values)
            else:
                space[param] = values
        return space

    def _create_objective_function(self, X, y):
        def objective(params):
            estimator = self.model.set_params(**params)
            score = -1.0 * np.mean(cross_val_score(estimator, X, y, cv=5, scoring='accuracy'))
            return score

        return objective

    def _convert_hyperopt_params(self, hyperopt_params):
        converted_params = {}
        for param, value in hyperopt_params.items():
            if isinstance(value, int):
                converted_params[param] = self.optimization_params[param][value]
            else:
                converted_params[param] = value
        return converted_params

    def _convert_genetic_params(self, genetic_params):
        converted_params = {}
        for param, value in genetic_params.items():
            if isinstance(value, int):
                converted_params[param] = self.optimization_params[param][value]
            else:
                converted_params[param] = value
        return converted_params

    @classmethod
    def display_results(cls):
        for result in cls.results:
            best_estimator, best_params = result
            print("Best Estimator:", best_estimator)
            print("Best Parameters:", best_params)
            print("---------------")

### Exampe for Decision Tree Regressor:

In [40]:
regressor_model = DecisionTreeRegressor()

# Define the optimization parameters
optimization_method = 'randomized'
optimization_params = {'max_depth': [None, 5, 10],
                       'min_samples_split': [2, 4],
                       'min_samples_leaf':[1],
                       'max_features':[None],
                       'random_state':[42]}

# Create an instance of ModelOptimizer
optimizer = ModelOptimizer(regressor_model, optimization_method, optimization_params)

# Perform the optimization
best_model, best_params = optimizer.optimize(X_train_scal, y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [41]:
ModelOptimizer.display_results()

Best Estimator: DecisionTreeRegressor(min_samples_split=4, random_state=42)
Best Parameters: {'random_state': 42, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': None}
---------------


In [42]:
best_model

DecisionTreeRegressor(min_samples_split=4, random_state=42)

In [43]:
best_params

{'random_state': 42,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': None,
 'max_depth': None}

In [44]:
best_model.fit(X_train_scal, y_train)
y_pred = best_model.predict(X_valid_scal)
mean_absolute_error(y_valid, y_pred)

0.05003863482549236

In [45]:
metric_dict = {"r2":r2_score,"mse":mean_squared_error,"mae":mean_absolute_error}

In [46]:
for metric in metric_dict:
    print(metric, metric_dict[metric](y_valid,y_pred))

r2 0.7470822730382136
mse 0.019314870014096378
mae 0.05003863482549236


In [47]:
b_test = best_model.predict(X_test_scal)

In [48]:
for metric in metric_dict:
    print(metric, metric_dict[metric](y_test,b_test))

r2 -0.6681768612053963
mse 0.10974709623138168
mae 0.23971372443902134
